In [1]:
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import requests
import json
import time
import re

In [2]:
df_kospi_filing_2011_2015 = pd.read_excel('C:\\Users\\sylim2357\\Documents\\SNU\\projects\\10-KPlanet\\kospi200_2011_2015.xlsx')

In [3]:
def make_sublist(text):
    batch = []
    batch_size = 5
    text_list = [text for text in re.split('\. | -', text) if (len(text)>8)]
    num_batches = int(np.ceil(len(text_list)/batch_size))
    for i in range(num_batches):
        batch.append('. '.join(text_list[batch_size*i:batch_size*(i+1)]))
    return batch

text_series = df_kospi_filing_2011_2015['Text']\
    .replace("\\n|(전자공시시스템 dart\.fss\.or\.kr)?(\n)?(Page \d{1,2})?(?=\s)", " ", regex=True)\
    .replace("\s\s+", " ", regex=True)
text_series_sublists = text_series.apply(make_sublist)

In [4]:
translated_text_list = []

In [5]:
text = ''
url = 'https://openapi.naver.com/v1/papago/n2mt'
headers = {
    "Content-Type": "application/json",
    "charset": "UTF-8",
    "X-Naver-Client-Id": "LfVavK978v9RUCxw5kKv",
    "X-Naver-Client-Secret": "9ikBjKPM6J",
}
data = {
    "source": "ko",
    "target": "en",
    "text": ''
}

for idx, text_sublist in text_series_sublists[:500].iteritems():
    print(idx)
    translated_text = []
    for text in text_sublist:
        time.sleep(np.random.randint(100,400)/101)
        data['text'] = text
        r = requests.post(url, data=json.dumps(data), headers=headers)
        translated_text.append(r.json()['message']['result']['translatedText'])
    translated_text_list.append('. '.join(translated_text))

0
1


KeyError: 'message'

In [19]:
textfile = open('../data/papago_translated_kospi200_2011_2015_index0.txt', 'wb')
textfile.write('. '.join([''.join(text.split('. ')) for text_list in translated_text_list for text in text_list ]).encode('utf-8'))
textfile.close()